# Understanding the Huang Crystal Plasticity UMAT

## Important subroutines

Below are some important subroutines in the UMAT which can be modified when altering the underlying constitutive models.

| Subroutine | Locations in UMAT | Description |
| --- | --- | --- |
| GSLPINT() | Line: 1905 | Critical resolved shear stress (CRSS) for each slip system |
| STRAINRATE() | Line: 1999 | The is the location of the power law being applied, e.g. $𝛾 ̇=𝛾_0 | \frac{𝜏}{𝑔}|^𝑛$ |
| LATENTHARDEN() | Line: 2112 | This is where your hardening model will reside, e.g. $ℎ_0  sech^2⁡| \frac{ℎ_0 𝛾}{(𝜏_𝑠−𝜏_0 )}|$ |
| ITERATION()| Line: 2414| Included here are the derivatives of the hardening models required to implement a Newton-Raphson solving capability |

## Modifications to UMAT (Taylor Expansion)

It is possible to modify and add your own versions of flow and hardening rules.
Some things to consider when doing this:

* We are ultimately solving for $\Delta \gamma^\alpha $, we must recognise that in order to do this:
    * Apply tangent modulus method: $\Delta \gamma^\alpha = \Delta t [(1-\theta){\dot{\gamma}_t}^\alpha + \theta {{\dot{\gamma}}^\alpha}_{t+\Delta t} ] $, where $\theta$ ranges from 0 to 1.
    * Calculate the shear strain at the next increment, ${{\dot{\gamma}}^\alpha}_{t+\Delta t}$, since the previous increment, ${\dot{\gamma}^\alpha}_t $
 

So we need to solve for shear strain for the next increment (${{\dot{\gamma}}^\alpha}_{t+\Delta t}$).

Using the Taylor expansion: ${\dot{\gamma}^\alpha}_{t+\Delta t} = {\dot{\gamma}^\alpha}_t + ({\dot{\gamma}_{\Delta t}}^\alpha)' $ :

where $({\dot{\gamma}_{\Delta t}}^\alpha)' $ is the derivative of the flow rule.

Given the followig flow rule:
${\dot{\gamma}}^\alpha = {\dot{\gamma_0}^\alpha} \bigg(\frac{|\tau^\alpha - X^\alpha|}{g^\alpha}\bigg) sgn(\tau^\alpha -X^\alpha) $
* $\dot{\gamma_{\Delta t}}^\alpha$ is a function of $\tau^\alpha$, $g^\alpha$, and $X^\alpha$
* Taking the derivative of this, results in the following partial derivatives:
$(\dot{\gamma_\Delta}^\alpha)' = \frac{\partial \dot{\gamma}(\tau^\alpha, g^\alpha, X^\alpha)}{\partial \tau^\alpha} \Delta \tau^\alpha + \frac{\partial \dot{\gamma}(\tau^\alpha, g^\alpha, X^\alpha)}{\partial g^\alpha} \Delta g^\alpha + \frac{\partial \dot{\gamma}(\tau^\alpha, g^\alpha, X^\alpha)}{\partial X^\alpha} \Delta X^\alpha $

Now lets take these partial derivatives of the flow rule provided.


$\frac{\partial \dot{\gamma}(\tau^\alpha, g^\alpha, X^\alpha)}{\partial \tau^\alpha} \Delta \tau^\alpha $

$ \dot{\gamma} = \gamma_0 \bigg ( \frac{|\tau - X|}{g} \bigg )^n $, $\frac{\partial \dot{\gamma}}{\partial \tau} = \frac{\gamma_0 n}{g}\bigg(\frac{|\tau-X|}{g} \bigg)^{n-1} $ modification line 2103

$\Delta \tau^\alpha = [L_{ijkl}\mu_{kl} + \omega_{ik}\sigma_{jk} + \omega_{jk}\sigma_{ik}] \bigg [\Delta \varepsilon_{ij} - \sum_{\beta} \mu_{ij} \Delta \gamma \bigg ] $

$\frac{\partial \dot{\gamma}(\tau^\alpha, g^\alpha, X^\alpha)}{\partial g^\alpha} \Delta g^\alpha$

$ \dot{\gamma} = \gamma_0 \bigg ( \frac{|\tau - X|}{g} \bigg )^n $, $\frac{\partial \dot{\gamma}}{\partial g} = \frac{-\gamma_0 n |\tau -X|}{g^2}\bigg(\frac{|\tau-X|}{g} \bigg)^{n-1} $ modification line 2103

$\Delta g = \sum_\beta \mathbf{h}_{\alpha \beta} \Delta \gamma^\beta$

$\frac{\partial \dot{\gamma}(\tau^\alpha, g^\alpha, X^\alpha)}{\partial X^\alpha} \Delta X^\alpha$

$ \dot{\gamma} = \gamma_0 \bigg ( \frac{|\tau - X|}{g} \bigg )^n $, $\frac{\partial \dot{\gamma}}{\partial X} = \frac{-\gamma_0 n}{g}\bigg(\frac{|\tau-X|}{g} \bigg)^{n-1} $ modification line 2103

$\Delta X = h \dot{\gamma} - h_D X |\dot{\gamma}|$

So now we have all components of:

$(\dot{\gamma_\Delta}^\alpha)' = \frac{\partial \dot{\gamma}(\tau^\alpha, g^\alpha, X^\alpha)}{\partial \tau^\alpha} \Delta \tau^\alpha + \frac{\partial \dot{\gamma}(\tau^\alpha, g^\alpha, X^\alpha)}{\partial g^\alpha} \Delta g^\alpha + \frac{\partial \dot{\gamma}(\tau^\alpha, g^\alpha, X^\alpha)}{\partial X^\alpha} \Delta X^\alpha $

So now we can substitute this back into the Taylor expansion:
$\dot{\gamma}_{t+\Delta t}^\alpha = \dot{\gamma_t}^\alpha + \frac{\partial \dot{\gamma}(\tau^\alpha, g^\alpha, X^\alpha)}{\partial \tau^\alpha} \Delta \tau^\alpha + \frac{\partial \dot{\gamma}(\tau^\alpha, g^\alpha, X^\alpha)}{\partial g^\alpha} \Delta g^\alpha + \frac{\partial \dot{\gamma}(\tau^\alpha, g^\alpha, X^\alpha)}{\partial X^\alpha} \Delta X^\alpha $


We can now substitute this into the Taylor Expansion
And then into the tangent modulus method:

$\Delta \gamma^\alpha = \Delta t [(1-\theta)\dot{\gamma^\alpha}_t + \theta \dot{\gamma^\alpha}_{t+\Delta t} ] $ :

$\Delta \gamma^\alpha = \Delta t \dot{\gamma^\alpha}_t - \Delta t \theta \dot{\gamma^\alpha}_t + \Delta t \theta \dot{\gamma^\alpha}_{t + \Delta t} $

$\therefore \Delta \gamma^\alpha = \Delta t \dot{\gamma^\alpha}_t - \Delta t \theta \dot{\gamma^\alpha}_t + \Delta t \theta \dot{\gamma^\alpha}_t  + \frac{\partial \dot{\gamma}(\tau^\alpha, g^\alpha, X^\alpha)}{\partial \tau^\alpha} \Delta \tau^\alpha + \frac{\partial \dot{\gamma}(\tau^\alpha, g^\alpha, X^\alpha)}{\partial g^\alpha} \Delta g^\alpha + \frac{\partial \dot{\gamma}(\tau^\alpha, g^\alpha, X^\alpha)}{\partial X^\alpha} \Delta X^\alpha $

$ \therefore \Delta t \dot{\gamma^\alpha}_t + \Delta t \theta \frac{\partial \dot{\gamma}}{\partial \tau}[L_{ijkl} \mu_{kl} + \omega_{ik}\sigma_{jk} + \omega_{jk}\sigma_{ik}] [\Delta \varepsilon_{ij} - \sum_\beta \mu_{ij}\Delta\gamma^\beta]  + \Delta t \theta \frac{\partial \dot{\gamma}}{\partial g} \sum_\beta \mathbf{h}_{\alpha \beta} \Delta \gamma^\beta + \Delta t \theta \frac{\partial \dot{\gamma}}{\partial X} [h \Delta t \gamma^\alpha - h_D X |\Delta \gamma^\alpha|]$

Rearrange for $\Delta \gamma^\alpha$:

$\sum_\beta \bigg[\Delta t \theta \frac{\partial \dot{\gamma}}{\partial \tau}[L_{ijkl} \mu_{kl} + \omega_{jk} \sigma_{jk} + \omega_{jk}\sigma_{ik}]\mu_{ij} - \Delta t \theta \frac{\partial \dot{\gamma}}{\partial g} \sum_\beta \mathbf{h}_{\alpha \beta} sign(\dot{\gamma_t}^\beta) - \Delta t \theta \frac{\partial \dot{\gamma}}{\partial X}( h \delta_{\alpha \beta} - h_D X \delta_{\alpha \beta}) + \delta_{\alpha \beta} \bigg ] \Delta \gamma^\beta = \Delta t \dot{\gamma_t}^\alpha + \Delta t \theta \frac{\partial \dot{\gamma}}{\partial \tau}[L_{ijkl} \mu_{kl} + \omega_{ik} \sigma_{jk} + \omega_{jk}\sigma_{ik}] \Delta \varepsilon_{ij}$

So now we need to solve for $\Delta \gamma$.  This is done in the UMAT using the subroutines LUDCMP and LUBKSB.



The location of this can be found in different locations in the UMAT:
$\sum_\beta \bigg[\Delta t \theta \frac{\partial \dot{\gamma}}{\partial \tau}[L_{ijkl} \mu_{kl} + \omega_{jk} \sigma_{jk} + \omega_{jk}\sigma_{ik}]\mu_{ij} - \Delta t \theta \frac{\partial \dot{\gamma}}{\partial g} \sum_\beta \mathbf{h}_{\alpha \beta} sign(\dot{\gamma_t}^\beta) - \Delta t \theta \frac{\partial \dot{\gamma}}{\partial X}( h \delta_{\alpha \beta} - h_D X \delta_{\alpha \beta}) + \delta_{\alpha \beta} \bigg ] \Delta \gamma^\beta $ 

Found between lines 932 - 956

$\Delta t \dot{\gamma_t}^\alpha + \Delta t \theta \frac{\partial \dot{\gamma}}{\partial \tau}[L_{ijkl} \mu_{kl} + \omega_{ik} \sigma_{jk} + \omega_{jk}\sigma_{ik}] \Delta \varepsilon_{ij}$ 

Found between lines 960- 989



## Modifications to UMAT (Newton-Raphson)

Now lets consider the Newton-Raphson component of the UMAT.
Again using the tangent modulus method, the function to iteratively solve:
    $R = \Delta t [(1-\theta) \dot{\gamma_t}^\alpha + \theta \dot{\gamma^\alpha}_{t + \Delta t}] - \Delta \gamma^\alpha $
    
We have used the Taylor expansion to approximate $\Delta \gamma^\alpha$ for the first iteration, which we can then use in the flow rule to approximate $\dot{\gamma^\alpha}_{t + \Delta t} $.

From now on, each value for $\Delta \gamma^\alpha $ can be approximated using the Netwon-Raphson method:
$\Delta \gamma^{k+1} = \Delta \gamma^k - \frac{\mathbf{R}}{\frac{\partial \mathbf{R}}{\partial \mathbf{\Delta \gamma}}} $, where $k$ is the Newton-Raphson iteration.

